In [1]:
from models.DAX import DAX_quantile_regression, DAX_baseline
from models.energy import energy_quantile_regression
from functions import get_energy, get_DAX, merge_submissions, check_submission
from functions.prepare_data import split_time
from functions.evaluation import evaluate_horizon
from tqdm import tqdm
from functions import evaluate_dax
from functions import naive_ensemble
import pandas as pd
#import minmaxscaler
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, date, timedelta


In [2]:
energydata = get_energy.get()

  0%|          | 0/263 [00:00<?, ?it/s]/Users/stephantimpe/PycharmProjects/probabilistic-forecasting-challenge/functions/get_energy.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 263/263 [00:27<00:00,  9.67it/s]


In [11]:
energydata.head()

,gesamt,weekday
date_time,,
2018-12-24 00:00:00,42.02925,0
2018-12-24 01:00:00,39.61025,0
2018-12-24 02:00:00,39.13875,0
2018-12-24 03:00:00,39.42100,0
2018-12-24 04:00:00,40.74775,0


Scale the data

In [14]:
# Creating a DataFrame
df = pd.DataFrame(energydata)

# Normalizing the 'gesamt' column
scaler = MinMaxScaler()
df['gesamt_normalized'] = scaler.fit_transform(df[['gesamt']])

df.head()  # Display the first few rows of the preprocessed DataFrame

,gesamt,weekday,gesamt_normalized
date_time,,,
2018-12-24 00:00:00,42.02925,0,0.504797
2018-12-24 01:00:00,39.61025,0,0.475743
2018-12-24 02:00:00,39.13875,0,0.470080
2018-12-24 03:00:00,39.42100,0,0.473470
2018-12-24 04:00:00,40.74775,0,0.489405


Split the data into train and test sets

In [39]:
# Assuming df has 'date_time' as its index and 'gesamt_normalized' as a column
# Extracting temporal features
df['hour'] = df.index.hour
df['day'] = df.index.day
df['month'] = df.index.month
# ... add other temporal features you find relevant

# Splitting the data into train and test sets chronologically
train_size = int(len(df) * 0.8)
train_df, test_df = df[:train_size], df[train_size:]

# Selecting the features and target for training
feature_columns = ['gesamt_normalized', 'hour', 'day', 'month'] # add other columns if needed
train_X = train_df[feature_columns].values
train_y = train_df['gesamt'].values.reshape(-1, 1)
test_X = test_df[feature_columns].values
test_y = test_df['gesamt'].values.reshape(-1, 1)

# Shapes of the training and testing data
train_X.shape, train_y.shape, test_X.shape, test_y.shape



((34859, 4), (34859, 1), (8715, 4), (8715, 1))

Train the LSTM model

In [40]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

# Assuming you have already prepared train_X, train_y, test_X, test_y as shown before

# Reshaping input data for LSTM [samples, time steps, features]
train_X_lstm = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
test_X_lstm = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))

# Designing the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(train_X_lstm.shape[1], train_X_lstm.shape[2])))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Training the model
model.fit(train_X_lstm, train_y, epochs=10, batch_size=1, verbose=1)

# You can then use model.predict to make predictions on your test set
# and evaluate the model's performance


Epoch 1/10


2023-12-13 15:02:16.871956: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-12-13 15:02:16.874055: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-12-13 15:02:16.874870: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

34859/34859 [==============================] - 18s 487us/step - loss: 71.6349
Epoch 2/10
34859/34859 [==============================] - 17s 490us/step - loss: 0.4715
Epoch 3/10
34859/34859 [==============================] - 17s 499us/step - loss: 0.3235
Epoch 4/10
34859/34859 [==============================] - 17s 480us/step - loss: 0.2340
Epoch 5/10
34859/34859 [==============================] - 18s 502us/step - loss: 0.2213
Epoch 6/10
34859/34859 [==============================] - 18s 508us/step - loss: 0.1881
Epoch 7/10
34859/34859 [==============================] - 19s 534us/step - loss: 0.1589
Epoch 8/10
34859/34859 [==============================] - 17s 474us/step - loss: 0.1624
Epoch 9/10
34859/34859 [==============================] - 16s 471us/step - loss: 0.1330
Epoch 10/10
34859/34859 [==============================] - 16s 453us/step - loss: 0.1339


In [41]:
from keras.layers import Dropout
from keras.models import load_model
from keras import backend as K
import numpy as np
import pandas as pd

# Assuming the model is already trained with dropout layers

# Function to enable dropout at inference time
def predict_with_dropout(model, input_data, n_iterations=100):
    predictions = []
    for _ in range(n_iterations):
        predictions.append(model.predict(input_data))
    return np.array(predictions)

# Using the function to make predictions with dropout
test_X_lstm = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))
predictions = predict_with_dropout(model, test_X_lstm)

# Calculate quantiles from predictions
quantiles = np.quantile(predictions, [0.025, 0.25, 0.5, 0.75, 0.975], axis=0)



216/273 [======================>.......] - ETA: 0s

2023-12-13 15:05:12.170637: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-12-13 15:05:12.171645: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-12-13 15:05:12.172098: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

273/273 [==============================] - 0s 310us/step


In [46]:
quantiles.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [44]:
import pandas as pd
# Example time horizons in hours
time_horizons = [36, 40, 44, 60, 64, 68]
quantile_levels = [0.025, 0.25, 0.5, 0.75, 0.975]

# Creating DataFrame
quantile_df = pd.DataFrame(quantiles.T, columns=[f'q{int(q*100)}' for q in quantile_levels])
quantile_df['horizon'] = time_horizons
quantile_df['forecast_date'] = pd.to_datetime('today').strftime('%Y-%m-%d')
quantile_df['target'] = 'energy'

# Rearranging columns
quantile_df = quantile_df[['forecast_date', 'target', 'horizon'] + [f'q{int(q*100)}' for q in quantile_levels]]

# Exporting to CSV
quantile_df.to_csv('energy_forecast_quantiles.csv', index=False)



ValueError: Must pass 2-d input. shape=(1, 8715, 5)

In [47]:
# Define the specific horizons (in number of time steps)
# Example: if each time step is 1 hour, for 36, 40, 44 hours ahead
horizons = [35, 39, 43, 59, 63, 67]  # Zero-based indexing

# Extract the relevant quantiles for these horizons
# This assumes the second dimension of 'quantiles' corresponds to different horizons
selected_quantiles = quantiles[:, horizons, 0]  # Shape: (num_quantiles, len(horizons))

# Creating the DataFrame
quantile_df = pd.DataFrame(selected_quantiles.T, columns=[f'q{int(q*100)}' for q in [0.025, 0.25, 0.5, 0.75, 0.975]])
quantile_df['horizon'] = [f'{h+1} hour' for h in horizons]  # h+1 because we subtracted 1 earlier
quantile_df['forecast_date'] = pd.to_datetime('today').strftime('%Y-%m-%d')
quantile_df['target'] = 'energy'

# Reordering the columns for clarity
quantile_df = quantile_df[['forecast_date', 'target', 'horizon'] + [f'q{int(q*100)}' for q in [0.025, 0.25, 0.5, 0.75, 0.975]]]

# Exporting to CSV
quantile_df.to_csv('energy_forecast_quantiles.csv', index=False)


In [48]:
quantile_df

,forecast_date,target,horizon,q2,q25,q50,q75,q97
0,2023-12-13,energy,36 hour,56.596985,56.596985,56.596985,56.596985,56.596985
1,2023-12-13,energy,40 hour,48.123363,48.123363,48.123363,48.123363,48.123363
2,2023-12-13,energy,44 hour,49.314907,49.314907,49.314907,49.314907,49.314907
3,2023-12-13,energy,60 hour,54.060722,54.060722,54.060722,54.060722,54.060722
4,2023-12-13,energy,64 hour,46.440258,46.440258,46.440258,46.440258,46.440258
5,2023-12-13,energy,68 hour,46.161129,46.161129,46.161129,46.161129,46.161129


In [55]:
energy_quantile_regression.energy_quantile_regression(energydata)

AttributeError: 'method_descriptor' object has no attribute 'today'

In [54]:
from models.energy import energy_quantile_regression

In [3]:
from models.energy import energy_LSTM

In [4]:
df_LSTM=energy_LSTM.build_and_forecast_lstm(energydata, '2024-01-05')

Epoch 1/10


2024-01-05 08:46:46.959820: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-05 08:46:46.961037: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-05 08:46:46.961748: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1103/1103 [==============================] - 2s 901us/step - loss: 1311.7590
Epoch 2/10
1103/1103 [==============================] - 1s 869us/step - loss: 169.5422
Epoch 3/10
1103/1103 [==============================] - 1s 854us/step - loss: 96.7665
Epoch 4/10
1103/1103 [==============================] - 1s 861us/step - loss: 96.0562
Epoch 5/10
1103/1103 [==============================] - 1s 863us/step - loss: 96.0247
Epoch 6/10
1103/1103 [==============================] - 1s 862us/step - loss: 96.0018
Epoch 7/10
1103/1103 [==============================] - 1s 890us/step - loss: 95.9794
Epoch 8/10
1103/1103 [==============================] - 1s 1ms/step - loss: 95.9848
Epoch 9/10
1103/1103 [==============================] - 1s 875us/step - loss: 95.9773
Epoch 10/10
1/1 [==============================] - 0s 262ms/step


2024-01-05 08:46:58.143471: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-05 08:46:58.144224: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-05 08:46:58.145100: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [5]:
df_LSTM

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-05,energy,36 hour,56.096050,56.096416,56.096115,56.096184,56.096355
1,2024-01-05,energy,40 hour,56.097294,56.097733,56.097378,56.097458,56.097645
2,2024-01-05,energy,44 hour,56.097637,56.098091,56.097725,56.097801,56.097996
3,2024-01-05,energy,60 hour,56.097008,56.097431,56.097092,56.097160,56.097355
4,2024-01-05,energy,64 hour,56.097572,56.098003,56.097652,56.097725,56.097931
5,2024-01-05,energy,68 hour,56.097713,56.098175,56.097805,56.097893,56.098080
